In [3]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
from io import BytesIO
import xmltodict
import requests
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")

from astropy.table import Table
from astropy.table import QTable
import astropy_healpix as ah
import astropy.units as u
from ligo.skymap.io import read_sky_map
from ligo.skymap.postprocess.cosmology import dVC_dVL_for_DL
from astropy.cosmology import Planck15 as cosmo, z_at_value

from ligo.gracedb.rest import GraceDb
g = GraceDb()

# O4a events

options to get all significan O4a mergers, or the 5 mergers considered in O4a paper

In [4]:
# use gracedb api call to get O4a significant superevents, but some of these have been retracted.

event_iterator = g.superevents('runid: O4a SIGNIF_LOCKED')
graceids = [superevent['superevent_id'] for superevent in event_iterator]

print (len(graceids), 'significant superevents in O4a')

89 significant superevents in O4a


In [5]:
responses = [g.superevent(id) for id in graceids]
data = [r.json() for r in responses]

In [4]:
def get_gcn_urls (ids, files):
    """
    remove retracted events
    get the most recent file for each superevent
    """ 

    superevent_files = [i['links']['files'] for i in files]

    event_files = [g.files(graceid).json() for graceid in ids]

    file = ['none' if any('etraction' in s for s in list(files))
            else id+'-5-Update.xml,0' if id+'-5-Update.xml,0' in list(files)
            else id+'-5-Update.xml' if id+'-5-Update.xml' in list(files)
            else id+'-4-Update.xml,0' if id+'-4-Update.xml,0' in list(files)
            else id+'-4-Update.xml' if id+'-4-Update.xml' in list(files)
            else id+'-3-Update.xml,0' if id+'-3-Update.xml,0' in list(files)
            else id+'-2-Update.xml,0' if id+'-2-Update.xml,0' in list(files)
            else id+'-4-Initial.xml,0' if id+'-4-Initial.xml,0' in list(files) 
            else id+'-3-Initial.xml,0' if id+'-3-Initial.xml,0' in list(files)
            else id+'-2-Initial.xml,0' if id+'-2-Initial.xml,0' in list(files)
            else id+'-2-Preliminary.xml,0' if id+'-2-Preliminary.xml,0' in list(files)
            else 'none' for files, id in zip(event_files, graceids)]

    urls = [i+j for i,j in zip(superevent_files, file)]

    [print(x) for x in urls if "none" in x]
    urls_save = [x for x in urls if "none" not in x]
    
    return(urls_save)

def get_params(xml_urls):
    """
    get superevent_id and skymap from event files
    """

    try:
        response = requests.get(xml_urls)
        dict=xmltodict.parse(response.text)
        superevent_id  = [item['@value'] for item in dict['voe:VOEvent']['What']['Param'] if item.get('@name') == 'GraceID'][0]
        skymap_url = [item['Param']['@value'] for item in dict['voe:VOEvent']['What']['Group'] if item.get('@name') == 'GW_SKYMAP'][0]
        classification = [item for item in dict['voe:VOEvent']['What']['Group'] if item.get('@name') == 'Classification']
        prob_bns = float([item['@value'] for item in classification[0]['Param'] if item.get('@name') == 'BNS'][0])  
        prob_nsbh = float([item['@value'] for item in classification[0]['Param'] if item.get('@name') == 'NSBH'][0])  
        skymap_response = requests.get(skymap_url)
        skymap_bytes = skymap_response.content
        skymap = Table.read(BytesIO(skymap_bytes))
        return superevent_id, skymap_bytes, skymap, prob_bns, prob_nsbh
    
    except:
        print (xml_urls)

In [7]:
#the printed urls are retracted events or my function failed to get the correct file

gcn_urls = get_gcn_urls (graceids, data)

https://gracedb.ligo.org/api/superevents/S231112ag/files/none
https://gracedb.ligo.org/api/superevents/S230830b/files/none
https://gracedb.ligo.org/api/superevents/S230808i/files/none
https://gracedb.ligo.org/api/superevents/S230715bw/files/none
https://gracedb.ligo.org/api/superevents/S230712a/files/none
https://gracedb.ligo.org/api/superevents/S230708bi/files/none
https://gracedb.ligo.org/api/superevents/S230622ba/files/none


In [ ]:
#returns event id, skymap bytes, skymap table, prob bns, prob nsbh    

params = [get_params(url) for url in gcn_urls]

In [9]:
# get subset of mergers that are potential BNS or NSBH

ns_events = [i for i in params if i[3] + i[4] > 0.1]
ids = [i[0] for i in ns_events] 
print(f'{len(ns_events)} events with prob > 0.1 for NSBH or BNS: {ids}') 

3 events with prob > 0.1 for NSBH or BNS: ['S230731an', 'S230627c', 'S230529ay']


In [5]:
# mergers triggered on in the O4a paper (https://arxiv.org/abs/2405.12403)

O4a_urls = ['https://gracedb.ligo.org/api/superevents/S230518h/files/S230518h-4-Update.xml,0',
            'https://gracedb.ligo.org/api/superevents/S230529ay/files/S230529ay-5-Update.xml,0',
            'https://gracedb.ligo.org/api/superevents/S230627c/files/S230627c-4-Update.xml,0',
            'https://gracedb.ligo.org/api/superevents/S230731an/files/S230731an-4-Update.xml,0',
            'https://gracedb.ligo.org/api/superevents/S231113bw/files/S231113bw-4-Update.xml,0',
            ]

In [6]:
O4a_params = [get_params(url) for url in O4a_urls]

In [37]:
skymaps = [param[2] for param in O4a_params]

# My volume calculation

In [ ]:
# get volume per pixel in 90% region of given skymap

#to do - 
# work in redshift not dl ?
# for cvs, get subset of volume that contains galactic plane
# look at stats: total volume, avg volume per pixel, max volume, min volume   

In [29]:
# adapted from https://github.com/lpsinger/ligo.skymap/blob/a8da314dcb078f7866f4178cbd523b9844cceae0/ligo/skymap/postprocess/crossmatch.py

def get_skymap_voxels(param, contour=0.9, cosmology=True):
    """
    Determine the volume per pixel in the 90% region of a skymap
    This volume is associated with a sky position and can be used with positionally nonuniform rates
    """
    # get specified countour of skymap, drop other pixels
    skymap = param[2]
    skymap.sort('PROBDENSITY', reverse = True)
    level, ipix = ah.uniq_to_level_ipix(skymap['UNIQ']) # ipix = sky location
    nside = ah.level_to_nside(level) # nside = multi-order pixel resolution
    pixel_area = ah.nside_to_pixel_area(ah.level_to_nside(level)) # pixel area in steradians
    prob = pixel_area * skymap['PROBDENSITY']
    cumprob = np.cumsum(prob)
    i = cumprob.searchsorted(contour)
    dA_sr = pixel_area[:i]
    dA_deg = dA_sr.to_value(u.deg**2) #areas in deg**2 per pixel for only pixels in countour
    #convert dA to mpc^2 ?

    # Calculate volume of each voxel, defined as the region within the
    # HEALPix pixel and contained within the two centric spherical shells
    # with radii (r - d_r / 2) and (r + d_r / 2).
    # r = 500 #mpc
    # d_r = 1000 #mpc
    # dV = (np.square(r) + np.square(d_r) / 12) * d_r * dA_deg.reshape(-1, 1) #naive euclidean volume from LIGO code

    # replacing above with my own Voxel - multiply volume of sphere radius 1000Mpc by fraction of pixel steradians / 4pi steradians in a sphere
    r = 1000 #mpc
    dV = 4/3 * np.pi * r**3 * dA_sr.reshape(-1, 1)/(4 * np.pi * u.sr) #euclidean volume

    # Calculate probability density per unit volume.
    if cosmology:
        dV *= dVC_dVL_for_DL(r) #comoving volume
    
    V = np.sum(dV)

    print(f'{param[0]} has total volume: {V} Mpc3') 
    return V, dV

In [30]:
voxels = [get_skymap_voxels(param) for param in O4a_params]

S230518h has total volume: 23146839.76344147 Mpc3
S230529ay has total volume: 1234160364.733098 Mpc3
S230627c has total volume: 4097930.0219590287 Mpc3
S230731an has total volume: 30102502.05441636 Mpc3
S231113bw has total volume: 86159440.48744766 Mpc3


In [26]:
# check euclidean volumes to validate assumption comoving volume is necessary at these redshifts

voxels_euc = [get_skymap_voxels(param, cosmology=False) for param in O4a_params]

S230518h has total volume: 46722498.16435014 Mpc3
S230529ay has total volume: 2491184799.5262804 Mpc3
S230627c has total volume: 8271778.34578611 Mpc3
S230731an has total volume: 60762683.431247875 Mpc3
S231113bw has total volume: 173915071.8265345 Mpc3


# get rates from volumes

add : poisson sampled distribution

In [34]:
transient_rates = {
    'SNIa': 2.35e4,  # per Gpc^3 per year
    'CCSN': 1.01e5,
    'SLSN': 5.6,
    'KN': 5e3,
    'GRB_on_axis': 1,
    'GRB_off_axis': 7,
    'CV': 1e6,  # Use a refined rate near the galactic plane
}

In [98]:
# probability density function for radii of events evenly distributed in sphere
def pdf_radii(num_events, sphere_radius = 1000):
    """
    Sample radii for events evenly distributed in a sphere.
    """
    u = np.random.uniform(0, 1, num_events)
    radii = sphere_radius * np.cbrt(u)  # Inverse transform sampling for r^2 PDF
    return radii

# get ras and decs
def random_ra_dec_skymap(num_events, skymap, contour=0.9):
    """
    Given skymap and number of events, return random ra, dec positions in 90% region
    """
    skymap.sort('PROBDENSITY', reverse = True)
    level, ipix = ah.uniq_to_level_ipix(skymap['UNIQ']) # ipix = sky location
    nside = ah.level_to_nside(level) # nside = multi-order pixel resolution
    pixel_area = ah.nside_to_pixel_area(ah.level_to_nside(level)) # pixel area in steradians
    prob = pixel_area * skymap['PROBDENSITY']
    cumprob = np.cumsum(prob)
    i = cumprob.searchsorted(contour)

    #get ra and dec in degrees associated with each pixel
    # fix this, temporarily only taking discrete ra and dec associated with each pixel
    ra, dec = ah.healpix_to_lonlat(ipix[:i], nside[:i])
    skymap_ra_deg = [r.deg for r in ra]
    skymap_dec_deg = [d.deg for d in dec]
    ras = np.random.choice(skymap_ra_deg, num_events)
    decs = np.random.choice(skymap_dec_deg, num_events)
    return ras, decs

# get rates for each transient given a skymap
def get_rates(transient_rates, voxel, time_window=7):
    num_events = []
    volume = voxel[0] * 10**-9 # convert to Gpc^3
    for transient, rate in transient_rates.items():
        events = rate * volume * time_window / 365.25
        num_events.append(float(events))

    print(f'{list(transient_rates.keys())} : {num_events}')
    return num_events

In [114]:
test = random_ra_dec_skymap(10, skymaps[0])

In [99]:
rates = [get_rates(transient_rates, voxel, time_window=7) for voxel in voxels]

['SNIa', 'CCSN', 'SLSN', 'KN', 'GRB_on_axis', 'GRB_off_axis', 'CV'] : [10.424791625150231, 44.80442358043291, 0.0024842056638655872, 2.21804077130856, 0.000443608154261712, 0.003105257079831984, 443.608154261712]
['SNIa', 'CCSN', 'SLSN', 'KN', 'GRB_on_axis', 'GRB_off_axis', 'CV'] : [555.8367693322235, 2388.9154767044497, 0.13245471950044474, 118.26314241111139, 0.023652628482222278, 0.16556839937555595, 23652.628482222277]
['SNIa', 'CCSN', 'SLSN', 'KN', 'GRB_on_axis', 'GRB_off_axis', 'CV'] : [1.845611194010295, 7.932201301916587, 0.0004398052207003256, 0.3926832327681478, 7.853664655362957e-05, 0.000549756525875407, 78.53664655362957]
['SNIa', 'CCSN', 'SLSN', 'KN', 'GRB_on_axis', 'GRB_off_axis', 'CV'] : [13.55745814634221, 58.26822437364098, 0.0032307134306177173, 2.884565563051534, 0.0005769131126103067, 0.004038391788272147, 576.9131126103067]
['SNIa', 'CCSN', 'SLSN', 'KN', 'GRB_on_axis', 'GRB_off_axis', 'CV'] : [38.8041833269956, 166.77542621389597, 0.009246954324730865, 8.256209218

In [106]:
def full_sample(transient_rates,voxel,skymap,time_window=7):
    """
    get sample of transients 
    
    Parameters:
    transient_rates: dictionary of transient types and rates in Gpc^3 / year
    voxels: list of tuples containing volume, volume per pixel, ra, dec for a given skymap
    skymap: table containing skymap data
    time_window: time window in days to calculate expected transients

    returns: 
    number of expected transients for each transient type, with sampled ra, dec, and distance (change this to redshift)
    """

    # get number of events for each transient type  
    num_events = get_rates(transient_rates, voxel, time_window)
    int_events = [round(num) for num in num_events]

    # sample ra and dec for each transients types
    radec = [random_ra_dec_skymap(num, skymap) for num in int_events]

    # get distances in Mpc from sphere with radius 1000Mpc
    dists = [pdf_radii(num) for num in int_events]
    
    return(num_events, radec, dists)

In [107]:
index = 0
print(f'getting sample for {O4a_params[index][0]}')
event0 = full_sample(transient_rates, voxels[index], skymaps[index], time_window=7)

getting sample for S230518h
['SNIa', 'CCSN', 'SLSN', 'KN', 'GRB_on_axis', 'GRB_off_axis', 'CV'] : [10.424791625150231, 44.80442358043291, 0.0024842056638655872, 2.21804077130856, 0.000443608154261712, 0.003105257079831984, 443.608154261712]


In [ ]:
# next steps: add galactic latitude consideration to rates

#plot
